In [91]:
import pickle
import re
from tqdm import tqdm
import binascii
import numpy as np
import random
import pandas as pd

In [121]:
news = pd.DataFrame()

In [122]:
news_temp = pd.read_csv("data/24.csv", index_col=0)

In [124]:
news_temp = pd.read_csv("data/lenta.csv", index_col=0)

In [126]:
news_temp = pd.read_csv("data/vesti.csv", index_col=0)

In [127]:
news = pd.concat([news, news_temp])

In [128]:
news.shape

(8368, 4)

In [132]:
news.iloc[5287]

date                                            2019.05.19
url            https://www.vestifinance.ru/articles/119411
title    Россия может принять решение о выдаче российск...
text     Прямой ЭфирЗапрет Китая на импорт отходов ожив...
Name: 22, dtype: object

In [100]:
news.tail()

,date,url,title,text
3014,2019.04.21,https://vesti.ru/doc.html?id=3148849&cid=680,"Тарасенко помог ""Сент-Луису"" разгромить ""Сан-Х...",StLouisBlues/Twitter В Сан-Хосе завершилась п...
3015,2019.04.21,https://vesti.ru/doc.html?id=3148847&cid=8,В Уганде перевернулась лодка: два человека пог...,"Global Look Press В воскресенье, 19 мая, на о..."
3016,2019.04.21,https://vesti.ru/doc.html?id=3148846&cid=680,Илья Воробьев: немного переудалялись в игре со...,Тема: twitter.com Главный тренер сборной Росси...
3017,2019.04.21,https://vesti.ru/doc.html?id=3148838&cid=5,"Штаб Зеленского обещает ""фишечки"" на церемонии...",Тема:В Киеве все выходные готовились к инаугур...
3018,2019.04.21,https://vesti.ru/doc.html?id=3148843&cid=8,Взрыв гранаты во время концерта в Колумбии: се...,Global Look Press На северо-востоке Колумбии ...


In [131]:
news.dropna(subset=["text"], inplace=True)

In [113]:
news = news.to_dict("records")

In [2]:
with open("data/all_news.pkl", 'rb') as f:
    news = pickle.load(f)

In [12]:
def get_shingles(words, n=2):
    for i in range (0,len(words)-n+1):
        yield ' '.join(words[i:i+n])

In [13]:
def clean(source):
    stop_symbols = '.,!?:;-\n\r()—'

    stop_words = ('это', 'как', 'так',
                  'и', 'в', 'над',
                  'к', 'до', 'не',
                  'на', 'но', 'за',
                  'то', 'с', 'ли',
                  'а', 'во', 'от',
                  'со', 'для', 'о',
                  'же', 'ну', 'вы',
                  'бы', 'что', 'кто',
                  'он', 'она')

    return [x for x in [y.strip(stop_symbols) for y in source.lower().split()] if x]

In [14]:
shingle_sets = {}
all_shingles = set()
i = 0
for document in tqdm(news):
    shingles_in_doc = set()
    clean_doc = clean(document['text'])
    shingles = {i for i in get_shingles(clean_doc, 5)}
    for shingle in shingles:
        crc = binascii.crc32(shingle.encode('utf-8')) & 0xffffffff
        shingles_in_doc.add(crc)
        all_shingles.add(crc)
    
    shingle_sets[i] = shingles_in_doc
    i += 1

100%|████████████████████████████████████| 8228/8228 [00:07<00:00, 1129.91it/s]


In [15]:
all_shingles = list(all_shingles)

In [16]:
docs_num = len(news)

In [17]:
sim_matrix = np.zeros((docs_num, docs_num))

In [ ]:
for i in tqdm(range(docs_num)):
    doc_1 = shingle_sets[i]
    for j in range(i+1, docs_num):
        doc_2 = shingle_sets[j]
        try:
            sim = len(doc_1.intersection(doc_2)) / len(doc_1.union(doc_2))
            sim_matrix[i][j] = sim
            sim_matrix[j][i] = sim
        except:
            sim_matrix[i][j] = 0
            sim_matrix[j][i] = 0

  4%|█▍                                     | 302/8228 [01:14<30:05,  4.39it/s]

In [25]:
threshold = 0.6

In [26]:
sims = []
i = 1
for j in range(i + 1, docs_num):
    estJ = sim_matrix[i][j]
    
    texts = []
    if estJ > threshold:
        texts.append(news[j]['text'])
        
    if len(texts) > 0:
        sims.append((news[i]['text'], texts))

In [32]:
max(sim_matrix[50])

0.01951219512195122

In [20]:
print(news[i]['body'])

Победитель президентских выборов на Украине Владимир Зеленский прокомментировал предложение президента России Владимира Путина о намерении в будущем предоставлять гражданам Украины российское гражданство в упрощенном порядке. Заявление опубликовано в Telegram-канале команды политика. По словам Зеленского, российские власти «зря тратят время» в попытках «соблазнить» граждан Украины своими паспортами. В то же время он не исключил возможности, что кому-то он может понадобиться «ради заработка или в попытке скрыться от уголовных расследований», или же вследствие «влияния пропаганды». Зеленский подчеркнул, что украинцы «прекрасно понимают», что значит быть гражданином России. По его мнению, это отсутствие «мирных протестов, конкурентных выборов, естественных прав и свобод человека». «Гражданство Украины — это свобода, достоинство и честь. Это то, что мы защитили и будем защищать. Украина не откажется от своей миссии служить примером демократии для постсоветских стран», — заключил Зеленский.

In [16]:
max_val = 0
max_i = 0
for i in range(i, docs_num):
    if max(sim_matrix[i]) > max_val:
        max_val = max(sim_matrix[i])
        max_i = i

In [17]:
max_val

0.6163793103448276

In [25]:
max_i

12702

In [26]:
max(sim_matrix[12702])

0.6163793103448276

In [28]:
for j in range(docs_num):
    estJ = sim_matrix[i][j]
    
    if estJ > threshold:
        print(j)
        print(news[j]['body'])
        print()

14317
Новый чрезвычайный и полномочный посол России в Белоруссии Дмитрий Мезенцев прибыл на свое место работы в Минск. Об этом сообщает ТАСС со ссылкой на пресс-службу российского посольства в Белоруссии. Мезенцев был назначен на пост чрезвычайного и уполномоченного посла России в Белоруссии 30 апреля, сразу после отставки Михаила Бабича, который также был специальным представителем президента России. СМИ писали, что решение об отставке Бабича было принято после неоднократных просьб, которые адресовал лично российскому лидеру Владимиру Путину президент Белоруссии Александр Лукашенко. С момента назначения Бабича на пост посла в августе 2018 года, белорусская сторона неоднократно выражала недовольство его деятельностью. В частности, Лукашенко обвинял дипломата в том, что он дезинформировал журналистов по вопросу стоимости совместного с Москвой проекта АЭС, а МИД Белоруссии заявлял, что Бабич разрушает отношения между двумя странами. Тем не менее в Кремле сообщали, что Москва была довольн

## MinHashing

In [17]:
num_hashes = 25
sim_matrix = np.zeros((docs_num, docs_num))

In [18]:
def build_new_hashes():
    new_list = []
    for i in range(num_hashes):
        r = random.sample(all_shingles, len(all_shingles))
        new_list.append(r)
        
    return new_list

In [19]:
def calculate_hash(matrix, shingles):
    for i in range(len(matrix)):
        if matrix[i] in shingles:
            return i

In [20]:
hashes = build_new_hashes()

In [21]:
signatures = []

In [22]:
for i in tqdm(range(docs_num)):
    signature = []
    shingles_set = shingle_sets[i]
    
    for m in hashes:
        cur_hash = calculate_hash(m, shingles_set)
        signature.append(cur_hash)
        
    signatures.append(signature)

100%|██████████████████████████████████████| 8228/8228 [07:33<00:00, 18.14it/s]


In [23]:
sim_matrix = np.zeros((docs_num, docs_num))

In [24]:
for i in tqdm(range(docs_num)):
    signature1 = signatures[i]
    for j in range(i + 1, docs_num):
        signature2 = signatures[j]

        count = 0
        for k in range(num_hashes):
            count += signature1[k] == signature2[k]

        sim_matrix[i][j] = (count / num_hashes)
        sim_matrix[j][i] = (count / num_hashes)

100%|██████████████████████████████████████| 8228/8228 [04:23<00:00, 31.23it/s]


In [25]:
threshold = 0.6

In [45]:
i

0

In [109]:
sims = []
duplicates = []
is_break = False
for i in range(docs_num):
    for j in range(i+1, docs_num):
        sim = sim_matrix[i][j]

        texts = []
        if sim > threshold:
            duplicates.append((i, j, sim))
#             texts.append(news[j]['text'])

#         if len(texts) > 0:
#             sims.append((news[i]['text'], texts))

In [110]:
duplicates

[(1295, 2617, 1.0),
 (1295, 3682, 1.0),
 (1295, 4353, 1.0),
 (1295, 5269, 1.0),
 (1295, 5368, 1.0),
 (1295, 5467, 1.0),
 (1295, 5566, 1.0),
 (1295, 5665, 1.0),
 (1295, 5764, 1.0),
 (1295, 5864, 1.0),
 (1295, 5964, 1.0),
 (1295, 6064, 1.0),
 (1295, 6164, 1.0),
 (1295, 6264, 1.0),
 (1295, 6365, 1.0),
 (1295, 6466, 1.0),
 (1295, 6567, 1.0),
 (1295, 6668, 1.0),
 (1295, 6769, 1.0),
 (1295, 6870, 1.0),
 (1295, 6971, 1.0),
 (1295, 7073, 1.0),
 (1295, 7175, 1.0),
 (1295, 7277, 1.0),
 (1295, 7380, 1.0),
 (1295, 7483, 1.0),
 (1295, 7586, 1.0),
 (1295, 7689, 1.0),
 (1295, 7794, 1.0),
 (1295, 7900, 1.0),
 (1295, 8006, 1.0),
 (1295, 8118, 1.0),
 (2304, 2387, 0.68),
 (2617, 3682, 1.0),
 (2617, 4353, 1.0),
 (2617, 5269, 1.0),
 (2617, 5368, 1.0),
 (2617, 5467, 1.0),
 (2617, 5566, 1.0),
 (2617, 5665, 1.0),
 (2617, 5764, 1.0),
 (2617, 5864, 1.0),
 (2617, 5964, 1.0),
 (2617, 6064, 1.0),
 (2617, 6164, 1.0),
 (2617, 6264, 1.0),
 (2617, 6365, 1.0),
 (2617, 6466, 1.0),
 (2617, 6567, 1.0),
 (2617, 6668, 1.0),

In [119]:
news[1295]

{'date': '2019/05/17',
 'url': 'https://lenta.ru/extlink/2019/05/17/guesswho/',
 'title': 'Это что за покемон',
 'text': 'None'}

In [120]:
news[2617]

{'date': '2019/05/08',
 'url': 'https://lenta.ru/extlink/2019/05/08/repinmyth/',
 'title': 'Вы его даже не представляете',
 'text': 'None'}

In [73]:
dupl = []

In [74]:
for i, j in duplicates:
    if news[i]['text'] != news[j]['text']:
        dupl.append((i, j))

In [75]:
dupl

[(1295, 5269),
 (1295, 5368),
 (1295, 5467),
 (1295, 5566),
 (1295, 5665),
 (1295, 5764),
 (1295, 5864),
 (1295, 5964),
 (1295, 6064),
 (1295, 6164),
 (1295, 6264),
 (1295, 6365),
 (1295, 6466),
 (1295, 6567),
 (1295, 6668),
 (1295, 6769),
 (1295, 6870),
 (1295, 6971),
 (1295, 7073),
 (1295, 7175),
 (1295, 7277),
 (1295, 7380),
 (1295, 7483),
 (1295, 7586),
 (1295, 7689),
 (1295, 7794),
 (1295, 7900),
 (1295, 8006),
 (1295, 8118),
 (2304, 2387),
 (2617, 5269),
 (2617, 5368),
 (2617, 5467),
 (2617, 5566),
 (2617, 5665),
 (2617, 5764),
 (2617, 5864),
 (2617, 5964),
 (2617, 6064),
 (2617, 6164),
 (2617, 6264),
 (2617, 6365),
 (2617, 6466),
 (2617, 6567),
 (2617, 6668),
 (2617, 6769),
 (2617, 6870),
 (2617, 6971),
 (2617, 7073),
 (2617, 7175),
 (2617, 7277),
 (2617, 7380),
 (2617, 7483),
 (2617, 7586),
 (2617, 7689),
 (2617, 7794),
 (2617, 7900),
 (2617, 8006),
 (2617, 8118),
 (3018, 3020),
 (3682, 5269),
 (3682, 5368),
 (3682, 5467),
 (3682, 5566),
 (3682, 5665),
 (3682, 5764),
 (3682, 58

In [85]:
news[5287]

{'date': '2019.05.19',
 'url': 'https://www.vestifinance.ru/articles/119411',
 'title': 'Россия может принять решение о выдаче российских паспортов жителям Приднестровья ',
 'text': 'Прямой ЭфирЗапрет Китая на импорт отходов оживляет перерабатывающую промышленность США11:22Чистая прибыль "Русагро" по МСФО в I квартале выросла в 7 раз11:19Доход на одного сотрудника: топ-25 компаний США11:19Федерация независимых профсоюзов России предложила обнулить ставку НДФЛ для бедных россиян11:15Россия может принять решение о выдаче российских паспортов жителям Приднестровья10:55МВФ загнал Украину в долговую яму10:44Владимир Зеленский вступил в должность президента Украины10:35Рубль укрепился к доллару и евро на старте торгов10:24Проект строительства газопровода "Северный поток-2" профинансирован уже на 80%10:20Сложный квартал: нацпроекты не выполнялись, инфляция выросла, доходы населения упали10:05Торговая война может снизить темпы роста ВВП Китая на 1 процентный пункт09:58Fesco и РЖД наладят перев

In [84]:
news[6367]

{'date': '2019.05.08',
 'url': 'https://www.vestifinance.ru/articles/119315',
 'title': 'Торговая война между США и Китаем вызывает отток капитала с развивающихся рынков Азии',
 'text': 'Прямой ЭфирЗапрет Китая на импорт отходов оживляет перерабатывающую промышленность США11:22Доход на одного сотрудника: топ-25 компаний США11:19Чистая прибыль "Русагро" по МСФО в I квартале выросла в 7 раз11:19Федерация независимых профсоюзов России предложила обнулить ставку НДФЛ для бедных россиян11:15Россия может принять решение о выдаче российских паспортов жителям Приднестровья10:55МВФ загнал Украину в долговую яму10:44Владимир Зеленский вступил в должность президента Украины10:35Рубль укрепился к доллару и евро на старте торгов10:24Проект строительства газопровода "Северный поток-2" профинансирован уже на 80%10:20Сложный квартал: нацпроекты не выполнялись, инфляция выросла, доходы населения упали10:05Торговая война может снизить темпы роста ВВП Китая на 1 процентный пункт09:58Fesco и РЖД наладят п

In [36]:
len(sims)

162288

In [38]:
sims[162287]

('Прямой ЭфирСтраны ОПЕК+ обсуждают увеличение добычи11:53Запрет Китая на импорт отходов оживляет перерабатывающую промышленность США11:22Чистая прибыль "Русагро" по МСФО в I квартале выросла в 7 раз11:19Доход на одного сотрудника: топ-25 компаний США11:19Федерация независимых профсоюзов России предложила обнулить ставку НДФЛ для бедных россиян11:15Россия может принять решение о выдаче российских паспортов жителям Приднестровья10:55МВФ загнал Украину в долговую яму10:44Владимир Зеленский вступил в должность президента Украины10:35Рубль укрепился к доллару и евро на старте торгов10:24Проект строительства газопровода "Северный поток-2" профинансирован уже на 80%10:20Сложный квартал: нацпроекты не выполнялись, инфляция выросла, доходы населения упали10:05Торговая война может снизить темпы роста ВВП Китая на 1 процентный пункт09:58Fesco и РЖД наладят перевозку грузов из стран АТР по Транссибу в Европу09:54Компании РФ выбирают оборудование Китая и Индии вместо европейского09:34Цепная реакци